# Load the JSON into dataframes

In [5]:
from azure.storage.blob import ContainerClient
from ipython_secrets import *
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
sas = os.environ.get('AZURE_SAS')

if sas is None:
    sas = get_secret('AZURE_SAS')
    os.putenv('AZURE_SAS', sas)
    os.environ['AZURE_SAS'] = sas

# Instantiate a new ContainerClient
container_client = ContainerClient.from_container_url(sas)

In [3]:
# So far we have 4 large JSON files with recipes. All-recipes one is a bit of a mess, we need to clean it later.
recipes = ["processed/allrecipes/allrecipes-recipes.json","processed/bbc/bbccouk-recipes.json","processed/cookstr/cookstr-recipes.json","processed/epicurious/epicurious-recipes.json"]

raw = []
for i in recipes:
    blob_client = container_client.get_blob_client(i)
    download_stream = blob_client.download_blob()
    df = pd.read_json(download_stream.readall(),lines=True, encoding="Latin-1")
    raw.append(df)
    print("Loaded " + i + ". We have " + str(df.columns.tolist()) + "\n")

Loaded processed/allrecipes/allrecipes-recipes.json. We have ['author', 'cook_time_minutes', 'description', 'error', 'footnotes', 'ingredients', 'instructions', 'photo_url', 'prep_time_minutes', 'rating_stars', 'review_count', 'time_scraped', 'title', 'total_time_minutes', 'url']

Loaded processed/bbc/bbccouk-recipes.json. We have ['chef', 'chef_id', 'cooking_time_minutes', 'description', 'error', 'ingredients', 'instructions', 'instructions_detailed', 'photo_url', 'preparation_time_minutes', 'program', 'program_id', 'serves', 'time_scraped', 'title', 'total_time_minutes', 'url']

Loaded processed/cookstr/cookstr-recipes.json. We have ['chef', 'comment_count', 'contributors', 'cookbook', 'cookbook_publisher', 'cooking_method', 'copyright', 'cost', 'course', 'date_modified', 'description', 'dietary_considerations', 'difficulty', 'error', 'ingredients', 'ingredients_detailed', 'instructions', 'kid_friendly', 'make_ahead', 'makes', 'meal', 'occasion', 'photo_credit_name', 'photo_credit_si

## Lazy loading mechanism to avoid Blob storage

In [6]:
with open('interim.pkl', 'wb') as f:
    pickle.dump(raw, f)

In [7]:
with open('interim.pkl', 'rb') as f:
    raw = pickle.load(f)

## Clean up raw data

In [8]:
################################ Allrecipes Recipes ################################
allrecipes = raw[0]

# change name of rating-stars to rating, and drop minutes from time variables
allrecipes.rename({'rating_stars':'rating', 
                   'cook_time_minutes':'cook_time',
                   'prep_time_minutes':'prep_time',
                   'total_time_minutes':'total_time'}, axis=1, inplace=True)

# Fix some formatting
allrecipes.description = allrecipes.description.str.replace('[','').str.replace(']','')
allrecipes.footnotes = allrecipes.footnotes.apply(str).str.replace('[','').str.replace(']','')
allrecipes.footnotes = allrecipes.footnotes.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)
allrecipes.description = allrecipes.description.replace(r'\s+( +\.)|#',np.nan,regex=True).replace('',np.nan)


# Drop duplicates, somebody REALLY likes pizza
allrecipes = allrecipes[allrecipes['title'] !="Johnsonville® Three Cheese Italian Style Chicken Sausage Skillet Pizza"]

################################ BBC CO UK Recipes ################################
bbcrecipes = raw[1]

bbcrecipes = bbcrecipes.drop(['chef_id', 'instructions_detailed', 'program_id'], 1)

bbcrecipes.rename({'cooking_time_minutes':'cook_time',
                   'preparation_time_minutes':'prep_time',
                   'total_time_minutes':'total_time','serves':'makes',
                   'chef': 'author','program':'tag'}, axis=1, inplace=True)

################################ COOKSTR Recipes ################################
cookstrecipes = raw[2]

drop_cols = ['contributors', 'cookbook', 'cookbook_publisher', 'cooking_method', 'cost', 'course', 'dietary_considerations',
             'difficulty', 'meal', 'occasion', 'taste_and_texture', 'type_of_dish','rating_count', 'comment_count', 'copyright',
             'date_modified','ingredients_detailed','kid_friendly','make_ahead','photo_credit_name','photo_credit_site']
cookstrecipes = cookstrecipes.drop(drop_cols, 1)

# Rename rating
cookstrecipes.rename({'rating_value':'rating','chef':'author'}, axis=1, inplace=True)

# Fill missing ratings with 0
cookstrecipes.rating.fillna(0,inplace=True)

################################ epicurious check ################################
epicuriousrecipes = raw[3]

# Rename columns
epicuriousrecipes.rename({'prepSteps':'instructions', 
                          'aggregateRating':'rating', 
                          'reviewsCount':'review_count','author':'chef'}, axis=1, inplace=True)

# Change rating scale to 0-5
epicuriousrecipes.rating = epicuriousrecipes.rating*(5/4)
epicuriousrecipes.rename(columns={'dateCrawled':'time_scraped', 'hed':'title','dek':'description'}, inplace=True)

# Drop 100 missing ingredient recipes and other columns
epicuriousrecipes = epicuriousrecipes.dropna(0)
drop_cols = ['id', 'photoData', 'pubDate', 'type']
epicuriousrecipes = epicuriousrecipes.drop(drop_cols,1)
epicuriousrecipes.url = "https://www.epicurious.com" + epicuriousrecipes.url

epicuriousrecipes['author'] = [row[0]['name'] if len(row) > 0 else "" for row in epicuriousrecipes['chef']]

################################ Make full dataframe ################################
dfs = [allrecipes, bbcrecipes, cookstrecipes, epicuriousrecipes]
df_all = pd.concat(dfs,0,sort=True,ignore_index=True)

# Trash some columns because the data is not very reliable or interesting
df_total = df_all.drop(['willMakeAgainPct','chef','tag','error'], 1)
df_total = df_total.replace(0.0,np.nan)

df_total.rename({'review_count':'reviews','time_scraped':'scraped'})

nulls = df_total.isnull().sum(axis=0)

print(nulls.apply(lambda x: str(x)+str(' missing')))

print("\nDone!\nFull recipe dataset is ready!")
print("Final shape: {:,} rows with {} columns".format(df_total.shape[0], df_total.shape[1]))

author            1803 missing
cook_time        64972 missing
description         28 missing
footnotes       108832 missing
ingredients          0 missing
instructions         0 missing
makes           129976 missing
photo_url        42993 missing
prep_time        76477 missing
rating           40051 missing
review_count     41296 missing
time_scraped         0 missing
title                0 missing
total_time       48334 missing
url                  0 missing
dtype: object

Done!
Full recipe dataset is ready!
Final shape: 144,551 rows with 15 columns


# Tidying up the ingredients

We need to clean up the ingredients. This has to be done in two ways:
* Split them up into quantities and items in a dict
* Convert them from list into dict

In [9]:
fixed_ingredients = df_total.copy()

def fix_ingredient(ingredients):
    return [{'name': 'lemonjuice', 'qty': 5,'unit': 'tbsp', 'comment': 'fresh', 'fulltext': item} for item in ingredients]

# This is not really a fix obviously, need to write this still :-)
# TODO
fixed_ingredients['ingredients'] = [fix_ingredient(row) for row in df_total['ingredients']]

fixed_ingredients

,author,cook_time,description,footnotes,ingredients,instructions,makes,photo_url,prep_time,rating,review_count,time_scraped,title,total_time,url
0,Stephanie,25.0,I just started adding my favorite things to ba...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Preheat oven to 400 degrees F (205 degrees C)...,NaN,http://images.media-allrecipes.com/userphotos/...,55.0,4.32,46.0,1498204021,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
1,Stephanie,NaN,These are great as an appetizer or served alon...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...","[Combine parmesan cheese, pepper and garlic po...",NaN,http://images.media-allrecipes.com/userphotos/...,NaN,4.18,44.0,1498204301,Crispy Cheese Twists,NaN,http://allrecipes.com/Recipe/6663/
2,Kathy,NaN,This is the best bread recipe. Light and fluf...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Melt margarine in hot water. Add sugar and sa...,NaN,http://images.media-allrecipes.com/userphotos/...,NaN,3.65,168.0,1498206088,Mom's Yeast Rolls,NaN,http://allrecipes.com/Recipe/6665/
3,Mary E. Crain,NaN,A Southern delight with a 'more-ish' taste. ...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Combine sugar and oil; beat well. Add eggs an...,NaN,http://images.media-allrecipes.com/userphotos/...,NaN,4.70,344.0,1498209193,Sweet Potato Bread I,NaN,http://allrecipes.com/Recipe/6666/
4,DeeDee,45.0,You can make these into braids or buns. Frost ...,"""Cook's Note:"", 'I made 1 braid, 12 cinnamon b...","[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Stir butter and 1 teaspoon sugar into the hot...,NaN,http://images.media-allrecipes.com/userphotos/...,20.0,4.71,6.0,1498205251,Orange Buns,170,http://allrecipes.com/Recipe/6668/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144546,Daniel Shumski,NaN,Buttering the bread before you waffle it ensur...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...","[Preheat the waffle iron on low., Spread a thi...",NaN,NaN,NaN,NaN,NaN,1498857706,Waffled Ham and Cheese Melt with Maple Butter,NaN,https://www.epicurious.com/recipes/food/views/...
144547,Daniel Shumski,NaN,"Spread this easy compound butter on waffles, p...",NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Combine the ingredients in a medium-size bowl...,NaN,NaN,NaN,NaN,NaN,1498857726,Maple Butter,NaN,https://www.epicurious.com/recipes/food/views/...
144548,Daniel Shumski,NaN,Leftover mac and cheese is not exactly one of ...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Preheat the oven to 375°F. Butter a 9x5-inch ...,NaN,NaN,NaN,NaN,NaN,1498857706,Waffled Macaroni and Cheese,NaN,https://www.epicurious.com/recipes/food/views/...
144549,Kat Odell,NaN,A classic Mexican beer cocktail you can sip al...,NaN,"[{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...",[Place about 1/4 cup salt on a small plate. Ru...,NaN,NaN,NaN,NaN,NaN,1498857714,Classic Michelada,NaN,https://www.epicurious.com/recipes/food/views/...


In [10]:
blob_client = container_client.get_blob_client("total.csv")
blob_client.upload_blob(fixed_ingredients.to_csv(), blob_type="BlockBlob", overwrite=True)

blob_client = container_client.get_blob_client("total.json")
blob_client.upload_blob(fixed_ingredients.to_json(orient='records'), blob_type="BlockBlob", overwrite=True)


{'etag': '"0x8D780E7E4C74960"',
 'last_modified': datetime.datetime(2019, 12, 14, 22, 49, 36, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\xcf\xd5\x1eRXG\x00\xe6'),
 'client_request_id': '00909452-1ec4-11ea-856c-000d3a65bc03',
 'request_id': 'e8d2fafd-801e-0051-36d0-b239a3000000',
 'version': '2019-02-02',
 'date': datetime.datetime(2019, 12, 14, 22, 49, 35, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'error_code': None}

# Time to explore

In [11]:
fixed_ingredients[fixed_ingredients["title"]=="Pico de Gallo"]["ingredients"]

60912     [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
106688    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
107208    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
107209    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
107210    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
107212    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
107213    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
110832    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
114912    [{'name': 'lemonjuice', 'qty': 5, 'unit': 'tbs...
Name: ingredients, dtype: object